# Prepare synthetic data (Vitals)
It is going to be much easier to develop if you have realistic synthetic data.
Here we take a sql query that generates a single tabular output.
We run that query against the live identifiable data once.
We then use the [Synthetic Data Vault](https://sdv.dev/SDV/index.html) to prepare a synthetic model of those data.
The code below serves as a vignette for that process but will need adjusting to match the exact contents of the original query.

More complex examples that include multiple tables with joins and dependencies are also possible.

This notebook should be run interactively just once

## Set-up, query and return the data as datafame
The query lives in `./src/api/sitrep/` where `.` represents the project root.
If you run this JupyterNotebook using the local **Makefile** and `make run` then that query will be automatically copied here.

So first steps should be (from _this_ directory)
```sh
make mock1build
make mock2copyin
make mock3run
```
then navigate to http://uclvlddpragae07:8091/lab/tree/steve/work/synth_test_data_vitals.ipynb

In [1]:
import os
import pandas as pd
import numpy as np
import requests

from pathlib import Path
import sqlalchemy as sa

In [2]:
# Construct the PostgreSQL connection
emap_host = os.getenv('EMAP_DB_HOST')
emap_user = os.getenv('EMAP_DB_USER')
emap_passwd = os.getenv('EMAP_DB_PASSWORD')

emapdb_engine = sa.create_engine(f'postgresql://{emap_user}:{emap_passwd}@{emap_host}:5432/uds')

In [3]:
# Read the sql file into a query 'q' and the query into a dataframe
q = Path('vitals.sql').read_text()

In [4]:
df = pd.read_sql_query(sa.text(q), emapdb_engine)

Now rather than using a SQL query, this time we are going to use an existing API to populate a dataframe

In [5]:
df.shape

(4401, 16)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4401 entries, 0 to 4400
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   visit_observation_id  4401 non-null   int64              
 1   ob_tail_i             4401 non-null   int64              
 2   observation_datetime  4401 non-null   datetime64[ns, UTC]
 3   id_in_application     4401 non-null   object             
 4   value_as_real         2285 non-null   float64            
 5   value_as_text         2106 non-null   object             
 6   unit                  574 non-null    object             
 7   mrn                   4401 non-null   object             
 8   lastname              4401 non-null   object             
 9   firstname             4401 non-null   object             
 10  sex                   4401 non-null   object             
 11  date_of_birth         4401 non-null   object             
 12  bed_ad

## Generate a synthetic version of the real data

Use the table above to generate the metadata you need for the synthetic data

https://sdv.dev/SDV/user_guides/relational/relational_metadata.html#relational-metadata

In [7]:
# Minimal imports
from sdv import Metadata, SDV
from sdv.constraints import FixedCombinations

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SDV does not handle timezones nicely so remove

In [8]:
# PostgreSQL returns datetimes with tz info which sdv does not seem to be able to handle
def remove_timezone(df, col_name: str) -> pd.DataFrame:
    """sdv does not like timezones"""
    df[col_name] = pd.to_datetime(df[col_name], utc=True).dt.tz_localize(None)    
    return df

In [9]:
tz_cols = ['observation_datetime', 'bed_admit_dt']
for col in tz_cols:
    print(col)
    df = remove_timezone(df, col)

observation_datetime
bed_admit_dt


### Define PII that must be faked and not modelled

Define fields that contain PII and need faking (see the sketchy documentation [here](https://sdv.dev/SDV/developer_guides/sdv/metadata.html?highlight=pii#categorical-fields-data-anonymization) and the [Faker Documentation](https://faker.readthedocs.io/en/master/providers.html) for a full list of providers. Here is a brief example that specifies Fakers for [name](https://faker.readthedocs.io/en/master/providers/faker.providers.person.html#faker.providers.person.Provider.name) and [date of birth](https://faker.readthedocs.io/en/master/providers/faker.providers.date_time.html#faker.providers.date_time.Provider.date_of_birth). Note that you must pass arguments to a faker as a list.

NB: sdv also doesn't always recognise the columns correctly. Here we specify data_of_birth explicitly as a date whilst working on the larger task of defining columns that contain PII. See [field details](https://sdv.dev/SDV/developer_guides/sdv/metadata.html#field-details)

Now a full specification for the Vitals data

In [10]:
fields = {
    'visit_observation_id' : {
        'type': 'id',
        'subtype': 'integer',
    },
    'date_of_birth': {
        'type': 'datetime',
        'format': '%Y-%m-%d',
        'pii': True,
        # the 'pii_category' key defines the Faker function name (method)
        'pii_category': "date_of_birth", 
    },
    'lastname': {
        'type': 'categorical',
        'pii': True,
        'pii_category': 'last_name'
    },
    'firstname': {
        'type': 'categorical',
        'pii': True,
        'pii_category': 'first_name'
    },
    'mrn': {
        'type': 'categorical',
        'pii': True,
        'pii_category': ['random_number', 8 ]
    },
}

Prepare the constraints

In [11]:
fixed_obs_constraint = FixedCombinations(
    column_names=['id_in_application', 'value_as_real', 'value_as_text', 'unit'],
    handling_strategy='transform',
)

Prepare the metadata

In [12]:
metadata = Metadata()
metadata.add_table(
    name='tabpid',
    data=df,
    fields_metadata=fields,
    primary_key = 'visit_observation_id',
)

In [13]:
# Inspect the conversion that metadata.add_table did to the dataframe that you loaded
metadata.get_table_meta('tabpid')

{'fields': {'visit_observation_id': {'type': 'id', 'subtype': 'integer'},
  'date_of_birth': {'type': 'datetime',
   'format': '%Y-%m-%d',
   'pii': True,
   'pii_category': 'date_of_birth'},
  'lastname': {'type': 'categorical',
   'pii': True,
   'pii_category': 'last_name'},
  'firstname': {'type': 'categorical',
   'pii': True,
   'pii_category': 'first_name'},
  'mrn': {'type': 'categorical',
   'pii': True,
   'pii_category': ['random_number', 8]},
  'ob_tail_i': {'type': 'numerical', 'subtype': 'integer'},
  'observation_datetime': {'type': 'datetime'},
  'id_in_application': {'type': 'categorical'},
  'value_as_real': {'type': 'numerical', 'subtype': 'float'},
  'value_as_text': {'type': 'categorical'},
  'unit': {'type': 'categorical'},
  'sex': {'type': 'categorical'},
  'bed_admit_dt': {'type': 'datetime'},
  'dept_name': {'type': 'categorical'},
  'room_name': {'type': 'categorical'},
  'bed_hl7': {'type': 'categorical'}},
 'primary_key': 'visit_observation_id'}

Prepare the table(s)

In [14]:
tables = dict(tabpid=df)

Fit the model

In [15]:
from sdv.lite import TabularPreset
model = TabularPreset(name='FAST_ML', metadata=metadata.get_table_meta('tabpid'), constraints=[fixed_obs_constraint])
model.fit(df)
dfs=model.sample(df.shape[0])

In [16]:
dfs.loc[:30]

,visit_observation_id,ob_tail_i,observation_datetime,id_in_application,value_as_real,value_as_text,unit,mrn,lastname,firstname,sex,date_of_birth,bed_admit_dt,dept_name,room_name,bed_hl7
0,0,1,2022-06-25 08:40:10.955345152,9,18.0,NaN,NaN,60748233,Curtis,Scott,M,1970-07-23,2022-06-03 21:27:47.212632576,UCH T09 NORTH (T09N),BY12,BY02-08
1,1,1,2022-06-25 10:40:05.658662656,10,96.0,NaN,%,2133917,Murphy,Claudia,M,1938-07-27,2022-06-20 22:29:43.705277184,UCH T09 SOUTH (T09S),NON COVID MAJORS 04,BY01-25
2,2,1,2022-06-25 09:36:51.958138624,10,98.0,NaN,%,33639260,Smith,Joseph,F,2021-10-01,2022-06-30 04:58:43.720257024,UCH T08 NORTH (T08N),POOL ROOM,BY02-08
3,3,1,2022-06-25 08:31:53.785998336,8,74.0,NaN,NaN,94048643,Sheppard,Thomas,F,1940-06-25,2022-06-23 07:39:31.814795520,UCH T13 NORTH ONCOLOGY,POOL ROOM,ENDO
4,4,1,2022-06-25 11:06:12.618787840,9,18.0,NaN,NaN,39505791,Miller,Jessica,M,1932-09-12,2022-06-17 14:02:14.273678080,UCH T13 SOUTH (T13S),SR23,BY13-61
5,5,1,2022-06-25 12:56:37.650820352,9,18.0,NaN,NaN,70858191,Gilmore,Jose,M,1921-07-11,2022-06-28 12:45:57.446566656,UCH T13 NORTH ONCOLOGY,BY02,MAJ-CHAIR
6,6,1,2022-06-25 13:04:38.189788416,10,100.0,NaN,%,79195019,Armstrong,Denise,M,2019-12-07,2022-07-14 21:47:19.685906432,UCH T08 NORTH (T08N),BY11,BY03-15
7,7,1,2022-06-25 10:09:35.788353280,8,98.0,NaN,NaN,7068899,Ortiz,Anthony,F,1940-06-03,2022-07-04 15:24:30.724134400,UCH SDEC,POOL ROOM,BY02-10
8,8,1,2022-06-25 11:06:45.260291328,9,18.0,NaN,NaN,87869722,Jacobson,Kimberly,F,1990-02-20,2022-06-09 12:30:04.468867328,UCH T13 NORTH ONCOLOGY,SR21,BY13-62
9,9,1,2022-06-25 10:31:07.414492416,8,87.0,NaN,NaN,75531115,Dougherty,Angela,M,2007-03-09,2022-06-20 18:34:22.972912128,UCH T16 NORTH (T16N),POOL ROOM,UTC WR


Inspect the original data

In [17]:
# df.head()

Inspect the synthetic data

Finally transform admission_age_years to match the fake DoB

### Save the synthetic data

Options
- save the model and not the synthetic data (but then you need *sdv* to run the model)
- save the data (need some care with type conversions if you use csv etc.)

In [18]:
model.save('mock_vitals.pkl')

In [19]:
dfs.to_hdf('mock_vitals.h5', 'data')

/tmp/ipykernel_291/1530517875.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block3_values] [items->Index(['id_in_application', 'value_as_text', 'unit', 'mrn', 'lastname',
       'firstname', 'sex', 'date_of_birth', 'dept_name', 'room_name',
       'bed_hl7'],
      dtype='object')]

  dfs.to_hdf('mock_vitals.h5', 'data')


In [20]:
pd.read_hdf('mock_vitals.h5', 'data')

,visit_observation_id,ob_tail_i,observation_datetime,id_in_application,value_as_real,value_as_text,unit,mrn,lastname,firstname,sex,date_of_birth,bed_admit_dt,dept_name,room_name,bed_hl7
0,0,1,2022-06-25 08:40:10.955345152,9,18.0,NaN,NaN,60748233,Curtis,Scott,M,1970-07-23,2022-06-03 21:27:47.212632576,UCH T09 NORTH (T09N),BY12,BY02-08
1,1,1,2022-06-25 10:40:05.658662656,10,96.0,NaN,%,2133917,Murphy,Claudia,M,1938-07-27,2022-06-20 22:29:43.705277184,UCH T09 SOUTH (T09S),NON COVID MAJORS 04,BY01-25
2,2,1,2022-06-25 09:36:51.958138624,10,98.0,NaN,%,33639260,Smith,Joseph,F,2021-10-01,2022-06-30 04:58:43.720257024,UCH T08 NORTH (T08N),POOL ROOM,BY02-08
3,3,1,2022-06-25 08:31:53.785998336,8,74.0,NaN,NaN,94048643,Sheppard,Thomas,F,1940-06-25,2022-06-23 07:39:31.814795520,UCH T13 NORTH ONCOLOGY,POOL ROOM,ENDO
4,4,1,2022-06-25 11:06:12.618787840,9,18.0,NaN,NaN,39505791,Miller,Jessica,M,1932-09-12,2022-06-17 14:02:14.273678080,UCH T13 SOUTH (T13S),SR23,BY13-61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4396,4396,1,2022-06-25 11:51:31.085753088,9,18.0,NaN,NaN,43197988,King,Keith,M,1997-02-05,2022-05-24 06:26:49.256994816,UCH SDEC,BY08,15 SDEC
4397,4397,1,2022-06-25 09:00:39.943619840,6,98.8,NaN,NaN,91188743,Jones,John,F,1990-11-05,2022-06-18 12:21:54.787320320,UCH T09 NORTH (T09N),BY12,SR08-08
4398,4398,1,2022-06-25 09:44:35.095187456,NaN,NaN,NaN,NaN,72123766,Montgomery,Kimberly,F,1954-04-13,2022-05-27 11:07:03.489154304,UCH SDEC,BY04,SR08-08
4399,4399,1,2022-06-25 13:43:45.333158144,9,18.0,NaN,NaN,66666946,Baker,Joshua,M,1919-04-23,2022-06-01 20:59:25.361046528,UCH T01 ENHANCED CARE,SR10,SR14-14
